In [1]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [2]:
def load_data(hdf5_file, dataset_name_images, dataset_name_labels):
    with h5py.File(hdf5_file, 'r') as hf:
        data = np.array(hf[dataset_name_images])
        labels = np.array(hf[dataset_name_labels])
    return data, labels

# Loading model and Testing accuracy

In [9]:
LEC_loaded_model = tf.keras.models.load_model('Trained Model/LEC_model.keras')
LEC_loaded_model2 = tf.keras.models.load_model('Trained Model/LEC_model_with_layers.keras')
LEC_loaded_model3 = tf.keras.models.load_model('Trained Model/LEC_model_general.keras')
LEC_loaded_model4 = tf.keras.models.load_model('Trained Model/LEC_model_no_TL.keras')
MobileNetV3_S = tf.keras.models.load_model('Trained Model/MobileNetV3Small_with_TL.keras')
MobileNetV3_S2 = tf.keras.models.load_model('Trained Model/MobileNetV3Small_no_TL.keras')
MobileNetV3_L = tf.keras.models.load_model('Trained Model/Teacher_MobileNetV3Large_model.keras')




In [10]:
LEC_loaded_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model2.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model3.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model4.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
X_fer_test, y_fer_test = load_data('output files/fer2013_testing.h5', 'images', 'labels')
X_rafdb_test, y_rafdb_test = load_data('output files/rafdb_testing.h5', 'images', 'labels')
X_ck_test, y_ck_test = load_data('output files/rafdb_testing.h5', 'images', 'labels')

## Assessing Accuracy on FER-2013

In [12]:
score = LEC_loaded_model.evaluate(X_fer_test, y_fer_test, verbose=1)
score = LEC_loaded_model2.evaluate(X_fer_test, y_fer_test, verbose=1)
score = LEC_loaded_model3.evaluate(X_fer_test, y_fer_test, verbose=1)
score = LEC_loaded_model4.evaluate(X_fer_test, y_fer_test, verbose=1)
score = MobileNetV3_S.evaluate(X_fer_test, y_fer_test, verbose=1)
score = MobileNetV3_S2.evaluate(X_fer_test, y_fer_test, verbose=1)
score = MobileNetV3_L.evaluate(X_fer_test, y_fer_test, verbose=1)



225/225 ━━━━━━━━━━━━━━━━━━━━ 22s 90ms/step - accuracy: 0.6218 - loss: 1.1418
225/225 ━━━━━━━━━━━━━━━━━━━━ 20s 85ms/step - accuracy: 0.6062 - loss: 1.1849
225/225 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - accuracy: 0.5845 - loss: 1.1464
225/225 ━━━━━━━━━━━━━━━━━━━━ 28s 111ms/step - accuracy: 0.4357 - loss: 1.4586
225/225 ━━━━━━━━━━━━━━━━━━━━ 26s 109ms/step - accuracy: 0.5774 - loss: 1.1304
225/225 ━━━━━━━━━━━━━━━━━━━━ 26s 110ms/step - accuracy: 0.4687 - loss: 1.3298
225/225 ━━━━━━━━━━━━━━━━━━━━ 71s 306ms/step - accuracy: 0.6291 - loss: 1.1810


## Assessing Accuracy on RAF-DB

In [40]:
score = LEC_loaded_model.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)
score = LEC_loaded_model2.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)
score = LEC_loaded_model3.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)
score = MobileNetV3_S.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)
score = MobileNetV3_S2.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)
score = MobileNetV3_L.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)

96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.6048 - loss: 1.5531
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.6002 - loss: 1.6249
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.6252 - loss: 1.3993
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.6289 - loss: 1.2672
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 99ms/step - accuracy: 0.5620 - loss: 1.3620
96/96 ━━━━━━━━━━━━━━━━━━━━ 33s 340ms/step - accuracy: 0.6203 - loss: 1.4946


## Assessing Accuracy on CK+

In [41]:
score = LEC_loaded_model.evaluate(X_ck_test, y_ck_test, verbose=1)
score = LEC_loaded_model3.evaluate(X_ck_test, y_ck_test, verbose=1)
score = LEC_loaded_model2.evaluate(X_ck_test, y_ck_test, verbose=1)
score = MobileNetV3_S.evaluate(X_ck_test, y_ck_test, verbose=1)
score = MobileNetV3_S2.evaluate(X_ck_test, y_ck_test, verbose=1)
score = MobileNetV3_L.evaluate(X_ck_test, y_ck_test, verbose=1)

96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step - accuracy: 0.6048 - loss: 1.5531
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 102ms/step - accuracy: 0.6252 - loss: 1.3993
96/96 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.6002 - loss: 1.6249
96/96 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.6289 - loss: 1.2672
96/96 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.5620 - loss: 1.3620
96/96 ━━━━━━━━━━━━━━━━━━━━ 32s 333ms/step - accuracy: 0.6203 - loss: 1.4946
